### Digital Signature

#### What is Digital Signature?

Unlike encryption schemes which mainly focus on the security communications over an insecure network, digital signature solves another type of problem: verification.

The core problem of encryption is to transfer messages with integrity, that is to ensure that the original message wasn't eavesdropped nor manipulated. While the core problem of verification is to show a signal: the message received from sender(claimed) is really from sender itself, not from a third-party who claimed to be the sender.

#### How does digital signature works?

Digital signature scheme in which the owner of the private key ${Key}_{private}$ is able to create valid signatures, but knowledge of the public key ${Key}_{public}$ does not reveal the private key ${Key}_{private}$. 

Digital signatures are called as 'signature' because they are derived from the imitation of the real world.

Let's imagine a scenario where we open an account at a bank, and the bank requires us to sign the contract in person. Since handwriting and names are generally considered to be exclusive, a document signed in this way can be considered to have been read by the signer himself, i.e., it has been verified, which proves that the document belongs to the signer.

The essence of digital signatures lies in the fact that, under a private key encryption system, the private key itself forms a one-to-one mapping with the identity information of the key holder, while the identity of the private key can be verified by encrypting and decrypting operations with its corresponding public key.

In other words, suppose we have a pair of public and private keys, $Key_{public}, Key_{private}$, and an electronic document $D$ that needs to be signed. 

First, the document can be signed with the private key to obtain the signed document:

$D'=Signature\_Algorithm(D, Key_private)$

The recipient, after receiving the signed document $D'$, can then verify it using the sender's public key:

$Output=Verification\_Algorithm(D', Key_public)$

where the decryption result matches the original document only if  $Key\_public$ and $Key\_private$ belong to the same user, i.e., the sender.


#### RSA Digital Signature

##### Principle of RSA Digital Signature

The RSA digital signature technique is based on the RSA encryption algorithm, developed by Ron Rivest, Adi Shamir, and Leonard Adleman in 1977. The security of the RSA algorithm relies on the computational difficulty of factoring large composite numbers.

In the context of digital signatures, the RSA algorithm involves three steps: key generation, signature creation, and signature verification.

*Key Generation*

1. Choose two large prime numbers $p$ and $q$.
2. Compute their product $n=p\times q$, which will serve as the modulus for both the public and private keys.
3. Calculate Euler's totient function $\phi(n)=(p-1)(q-1)$.
4. Choose an integer $e$ that is less than $\phi(n)$ and coprime to $\phi(n)$ to be part of the public key. 65537 would be choose in common practice.
5. Compute the modular inverse of $e$ under $\phi(n)$, denoted as $d$, which will be part of the private key. That is, $de=1(\mod \phi(n))$

The public key consists of $(e,n)$, and the private key is $(d,n)$.

*Signature Creation*

The signature creation process involves encrypting the message $m$ using the private key $d$:

$signature = m^{d}\mod n$


*Signature Verification*

To verify the signature, use the public key $e$ and modulus $n$ to decrypt the signature:

$m={signature}^{e}\mod n$

If the decrypted message matches the original message, the signature is considered valid.

#### Elgamal Digital Signature

Elgamal digital signature is another asymmetric encryption-based signature scheme, proposed by Taher Elgamal in 1985. Unlike RSA, the Elgamal signature algorithm relies on the difficulty of the discrete logarithm problem.

*Key Generation*

1. Choose a large prime $p$ and a primitive root $g$.
2. Randomly select a private key $a$, where $a<p$.
3. Compute $g^a\mod p$, denoted as $A$.

The public key is $A$, and the private key is $a$.

*Signature Creation*

1. For each message $m$, choose a random number $k$, such that $k$ and $p−1$ are coprime.
2. Calculate $r=g^k\mod p$
3. Compute $s=(m-ar)k^{-1}\mod (p-1)$

The signature is $(r,s)$.

*Signature Verification*

1. Compute $v_1=g^m\mod p$.
2. Compute $v_2=A^r\times r^s\mod p$.

If $v_1==v_2$, then the signature is valid.

In particular, the use of elliptic curve groups leads to the Elliptic Curve Digital Signature Algorithm (ECDSA).

The advantage of using ECDSA lies in the fact that, the discrete logarithm problem for elliptic curves appears to be SIGNIFICANTLY HARDER than the discrete logarithm problem.

#### RSA Digital Signature Implementation

Here we show the process of RSA digital signature step by step.

1. generate_prime_candidate() will generate an odd integer randomly, which will be used in generating large prime $p, q$
2. generate_large_prime() will call generate_prime_candidate() with a given length, here we take default 1024.
3. generate_rsa_keys() will calculate $n, \phi(n)$, using default e=65537 and calculate $d$, returning 2-pair tuple (public_key, private_key).
4. rsa_sign() will use private key to sign message.
5. rsa_verify() will use public key to verify a signature, and check whether it is valid by comparing with message.

In [1]:
import random
from sympy import isprime, mod_inverse

def generate_prime_candidate(length):
    """ Generate an odd integer randomly """
    p = random.getrandbits(length)
    p |= (1 << length - 1) | 1
    return p

def generate_large_prime(length=1024):
    """ Generate a prime number of specified length """
    p = 4
    while not isprime(p):
        p = generate_prime_candidate(length)
    return p

def generate_rsa_keys(keysize=1024):
    """ Generate RSA public and private keys """
    p = generate_large_prime(keysize // 2)
    q = generate_large_prime(keysize // 2)
    n = p * q
    phi = (p - 1) * (q - 1)
    
    e = 65537  # Common choice for e
    while mod_inverse(e, phi) is None:
        e = random.randrange(1, phi)
    
    d = mod_inverse(e, phi)
    return ((e, n), (d, n))  # (public_key, private_key)

def rsa_sign(message, private_key):
    """ Create an RSA signature """
    d, n = private_key
    # Encrypting with private key
    message_int = int.from_bytes(message.encode('utf-8'), 'big')
    signature = pow(message_int, d, n)
    return signature

def rsa_verify(message, signature, public_key):
    """ Verify an RSA signature """
    e, n = public_key
    # Decrypting with public key
    message_int = int.from_bytes(message.encode('utf-8'), 'big')
    check_int = pow(signature, e, n)
    return message_int == check_int

We'll write usage example here, you can try to modify params' value to see if this works for differenct params.

In [2]:
public_key, private_key = generate_rsa_keys(1024)

In [3]:
message = "Hello, RSA!"

In [4]:
signature = rsa_sign(message, private_key)
print("Signature:", signature)

Signature: 19153648078298258310109365053158275292780177151960550221162068716317935735478208910475181092423638828363171186814278518579320391247565992303364823584769841847310507314337283711871129199728121322018065788622187872464422880588829712593767946433220248578092216490875233429780162498284783124935415826816966409712


In [5]:
is_valid = rsa_verify(message, signature, public_key)
print("Is the signature valid?", is_valid)

Is the signature valid? True


We also privide you an interactive widget. Run this cell down here to see your widget.

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output

message_input = widgets.Textarea(
    value='Hello, RSA!',
    description='Message:',
    continuous_update=False
)

key_length_slider = widgets.IntSlider(
    value=1024,
    min=512,
    max=4096,
    step=512,
    description='Key Length:',
    continuous_update=False
)

output = widgets.Output()

sign_button = widgets.Button(description="Sign")
verify_button = widgets.Button(description="Verify")

# Generate RSA keys
public_key, private_key = None, None
signature = None

def generate_rsa_keys_on_demand(change):
    global public_key, private_key
    key_size = change.new
    public_key, private_key = generate_rsa_keys(key_size)
    with output:
        clear_output()
        print(f"Generated new RSA keys with size: {key_size} bits")

key_length_slider.observe(generate_rsa_keys_on_demand, names='value')

def on_sign_button_clicked(b):
    global signature
    with output:
        clear_output()
        if public_key is None or private_key is None:
            print("Please generate the keys first.")
            return
        message = message_input.value
        signature = rsa_sign(message, private_key)
        print(f"Signature: {signature}")

def on_verify_button_clicked(b):
    with output:
        clear_output()
        if public_key is None:
            print("Please generate the keys first.")
            return
        if signature is None:
            print("No signature generated yet. Please sign the message first.")
            return
        message = message_input.value
        is_valid = rsa_verify(message, signature, public_key)
        print(f"Is the signature valid? {is_valid}")

sign_button.on_click(on_sign_button_clicked)
verify_button.on_click(on_verify_button_clicked)

display(message_input, key_length_slider, sign_button, verify_button, output)


Textarea(value='Hello, RSA!', continuous_update=False, description='Message:')

IntSlider(value=1024, continuous_update=False, description='Key Length:', max=4096, min=512, step=512)

Button(description='Sign', style=ButtonStyle())

Button(description='Verify', style=ButtonStyle())

Output()

#### Elgamal Digital Signature Implementation

Here we show the process of Elgamal digital signature step by step.
Note that the generate_prime_candidate() and generate_large_prime() are used again to produce $p$.

1. generate_elgamal_keys() will calculate every param inside the key pair: $p,g,a,A$, and return 2-pair tuple (public_key, private_key).
2. elgamal_sign() will use private key and p, gto sign message.
3. elgamal_verify() will use public key to verify a signature, and check whether it is valid by comparing with message.

In [7]:
import random
from sympy import isprime, mod_inverse
from math import gcd

def generate_prime_candidate(length):
    """ Generate an odd integer randomly """
    p = random.getrandbits(length)
    p |= (1 << length - 1) | 1
    return p

def generate_large_prime(length=1024):
    """ Generate a prime number of specified length """
    p = 4
    while not isprime(p):
        p = generate_prime_candidate(length)
    return p

def generate_elgamal_keys(keysize=256):
    """ Generate Elgamal public and private keys """
    p = generate_large_prime(keysize)
    g = random.randint(2, p - 2)
    a = random.randint(1, p - 2)
    A = pow(g, a, p)
    return ((p, g, A), a)  # (public_key, private_key)

def elgamal_sign(message, private_key, p, g):
    """ Create an Elgamal signature """
    a = private_key
    k = random.randrange(1, p - 1)
    while gcd(k, p - 1) != 1:
        k = random.randrange(1, p - 1)
    
    r = pow(g, k, p)
    k_inv = mod_inverse(k, p - 1)
    message_int = int.from_bytes(message.encode('utf-8'), 'big')
    s = (k_inv * (message_int - a * r)) % (p - 1)
    return (r, s)

def elgamal_verify(message, signature, public_key):
    """ Verify an Elgamal signature """
    p, g, A = public_key
    r, s = signature
    message_int = int.from_bytes(message.encode('utf-8'), 'big')
    v1 = pow(g, message_int, p)
    v2 = (pow(A, r, p) * pow(r, s, p)) % p
    return v1 == v2

Again, we'll write usage example here, you can try to modify params' value to see if this works for differenct params.

In [8]:
public_key, private_key = generate_elgamal_keys(256)

In [9]:
message = "Hello, Elgamal!"

In [10]:
signature = elgamal_sign(message, private_key, *public_key[:2])
print("Signature:", signature)

Signature: (67905536613399860576194549307068770144090721684496110017123651007610087275475, 15759443602806103022819007355413112588900091794560292304673866547072662271530)


In [11]:
is_valid = elgamal_verify(message, signature, public_key)
print("Is the signature valid?", is_valid)

Is the signature valid? True


Again, we provide an interactive widget.

In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output

message_input = widgets.Textarea(
    value='Hello, Elgamal!',
    description='Message:',
    continuous_update=False
)

key_length_slider = widgets.IntSlider(
    value=256,
    min=128,
    max=1024,
    step=64,
    description='Key Length:',
    continuous_update=False
)

output = widgets.Output()

sign_button = widgets.Button(description="Sign")
verify_button = widgets.Button(description="Verify")

# Generate Elgamal keys
public_key, private_key = None, None
signature = None 

def generate_keys(change):
    global public_key, private_key
    key_size = change.new
    public_key, private_key = generate_elgamal_keys(key_size)
    with output:
        clear_output()
        print(f"Generated new Elgamal keys with size: {key_size} bits")

key_length_slider.observe(generate_keys, names='value')

def on_sign_button_clicked(b):
    global signature
    with output:
        clear_output()
        if public_key is None or private_key is None:
            print("Please generate the keys first.")
            return
        message = message_input.value
        signature = elgamal_sign(message, private_key, *public_key[:2])
        print(f"Signature: {signature}")

def on_verify_button_clicked(b):
    with output:
        clear_output()
        if public_key is None:
            print("Please generate the keys first.")
            return
        if signature is None:
            print("No signature generated yet. Please sign the message first.")
            return
        message = message_input.value
        is_valid = elgamal_verify(message, signature, public_key)
        print(f"Is the signature valid? {is_valid}")

sign_button.on_click(on_sign_button_clicked)
verify_button.on_click(on_verify_button_clicked)

display(message_input, key_length_slider, sign_button, verify_button, output)


Textarea(value='Hello, Elgamal!', continuous_update=False, description='Message:')

IntSlider(value=256, continuous_update=False, description='Key Length:', max=1024, min=128, step=64)

Button(description='Sign', style=ButtonStyle())

Button(description='Verify', style=ButtonStyle())

Output()